In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [3]:

class TrafficDataset(Dataset):
    def __init__(self, directory,label_directory):
        
        self.files = [os.path.join(directory, f) for f in sorted(os.listdir(directory))] #[0:100]
        self.yfiles= [os.path.join(label_directory, f) for f in sorted(os.listdir(label_directory))] #[0:100]
        self.num_files = len(self.files)
        
        #self.sequences_per_file = 1000
        self.total_sequences = self.num_files # * self.sequences_per_file
        
    def __len__(self):
        return self.total_sequences
    
    def __getitem__(self, index):
        
        file_index = index 
        #sequence_idx = index % self.sequences_per_file
        
        data = np.load(self.files[file_index]) #Loads the file which contains our desired sequence
        label_data = np.load(self.yfiles[file_index])
        
        sequence = data # [sequence_idx]
        sequence = np.transpose(sequence, (2, 0, 1)) # To orient in the dimension with 3 channels being first
        
        y_sequence = label_data
        
        summed_y = np.sum(y_sequence,axis=0)
        majority_class = np.argmax(summed_y)
        label = np.zeros(13)
        label[majority_class] = 1
        
        return torch.tensor(sequence, dtype=torch.float32),torch.tensor(label, dtype=torch.float32)

In [4]:
print(f"Is CUDA available : {torch.cuda.is_available()}")

Is CUDA available : True


In [5]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        # Assuming the input shape is (batch_size, 3, 300, 42) after reshaping
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=1, padding=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=1, padding=2)
        self.fc1 = nn.Linear(1,128)
        self.fc2 = nn.Linear(128, 13)  # Output layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)  # Flatten starting from the 2nd dimension
            # Dynamically determine 'in_features' for the first fully connected layer
        if self.fc1.in_features == 1:  # Check if the placeholder value is still there
            num_features = x.shape[1]
            self.fc1 = nn.Linear(num_features, 128).to(x.device)  # Replace with the correct in_features
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
num_epochs  = 50
features_path = "data/dataset3/features/"
labels_path = "data/dataset3/labels/"
experiment_no = 1
n_classes = 13

dataset = TrafficDataset(features_path, labels_path)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True,num_workers=10)

model = CNNClassifier()


loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
correct_predictions_per_class = torch.zeros(n_classes, device=device)
actual_per_class = torch.zeros(n_classes, device=device)

In [10]:
print(f"Training started for {num_epochs} epochs")
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    start_time = time.time()
    total_predictions =0
    for i, (inputs, labels) in enumerate(dataloader):
        
        
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the device
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = loss_function(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        _, labels_indices = torch.max(labels, 1)
        total_samples += labels.size(0)
        
        predicted = predicted.to(labels_indices.device).long()
        # print(predicted) 
        labels_indices = labels_indices.to(predicted.device).long()
        # print(labels_indices)
        
        correct_predictions += (predicted == labels_indices).sum().item()
        total_predictions = total_predictions+32
        for class_index in range(n_classes):
            correct_predictions_per_class[class_index] += ((predicted == labels_indices) & (labels_indices == class_index)).sum()
            actual_per_class[class_index] += (labels_indices == class_index).sum()

        if i % 1 == 0:  # Log every 10 batches
            print(f"Epoch {epoch+1}, Batch {i+1}/{len(dataloader)}, Partial Loss: {running_loss/(i+1):.4f}, Correct Predictions: {correct_predictions}/{total_predictions}")
        # break
    # break
    epoch_time = time.time() - start_time
    epoch_loss = running_loss / len(dataloader)
    
    epoch_accuracy = (correct_predictions / total_samples) * 100


Training started for 50 epochs
Epoch 1, Batch 1/270, Partial Loss: 0.0025, Correct Predictions: 32/32
Epoch 1, Batch 2/270, Partial Loss: 0.0027, Correct Predictions: 64/64
Epoch 1, Batch 3/270, Partial Loss: 0.0024, Correct Predictions: 96/96
Epoch 1, Batch 4/270, Partial Loss: 0.0022, Correct Predictions: 128/128
Epoch 1, Batch 5/270, Partial Loss: 0.0021, Correct Predictions: 160/160
Epoch 1, Batch 6/270, Partial Loss: 0.0021, Correct Predictions: 192/192
Epoch 1, Batch 7/270, Partial Loss: 0.0022, Correct Predictions: 224/224
Epoch 1, Batch 8/270, Partial Loss: 0.0023, Correct Predictions: 256/256
Epoch 1, Batch 9/270, Partial Loss: 0.0021, Correct Predictions: 288/288
Epoch 1, Batch 10/270, Partial Loss: 0.0023, Correct Predictions: 320/320
Epoch 1, Batch 11/270, Partial Loss: 0.0023, Correct Predictions: 352/352
Epoch 1, Batch 12/270, Partial Loss: 0.0024, Correct Predictions: 384/384
Epoch 1, Batch 13/270, Partial Loss: 0.0024, Correct Predictions: 416/416
Epoch 1, Batch 14/270,